In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Command to convert .log.labeled files to .csv
# tr '\t' ',' < "(name).log.labeled" | cut -d ',' -f 1-22 > "(name).csv"

# Command to change directory
# cd /mnt/(path name)

# Replace first 7 lines of .csv file (until #types) with the header below
# timestamp,uid,src_ip,src_port,dest_ip,dest_port,protocol,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label

In [21]:
# Load the CSV files
benign = pd.read_csv(r"D:\AI Training\ML Model\CSV\Benign.csv")
torii = pd.read_csv(r"D:\AI Training\ML Model\CSV\Torii.csv")
mirai = pd.read_csv(r"D:\AI Training\ML Model\CSV\Mirai.csv")

# Add a label column for each dataset
benign['label'] = 0
torii['label'] = 1
mirai['label'] = 2

# Concatenate the datasets
data = pd.concat([benign, torii, mirai], axis=0)

# Shuffle the dataset
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows
data.head()

,timestamp,uid,src_ip,src_port,dest_ip,dest_port,protocol,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1.545411e+09,C1HrOb3gEhOYfqHxR5,192.168.1.195,56431,192.168.1.1,53,udp,dns,5.005399,78,...,S0,-,-,0,D,2,134,0,0,2
1,1.537529e+09,CQykLqpNaqHYbT3P3,192.168.2.3,46786,176.32.98.203,80,tcp,http,0.235654,135,...,SF,-,-,0,ShADafF,4,315,4,406,0
2,1.545411e+09,CYsX5h49x4DAaoGmR9,192.168.1.195,38326,192.168.1.1,53,udp,dns,0.131422,39,...,SF,-,-,0,Dd,1,67,1,131,2
3,1.545467e+09,CJXFEL3HKK9x0KzXZ6,192.168.1.195,65279,123.59.209.185,80,tcp,-,-,-,...,OTH,-,-,0,C,0,0,0,0,2
4,1.545415e+09,CazVLx1zrxkNjvrklj,192.168.1.195,52126,185.244.25.235,6667,tcp,irc,1.079634,75,...,S3,-,-,0,ShAdDaf,8,499,6,563,2


In [22]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(data['label']), y=data['label'])
class_weight_dict = dict(zip(np.unique(data['label']), class_weights))

print("Class weights:", class_weight_dict)

Class weights: {0: 5.144213216502373, 1: 2.9271839617741766, 2: 0.40584719521854973}
